In [2]:
#######################################################################
# This script was written by Nana Owusu, it is meant to visualize     #
# waveform information from XML files produced by the GE scanner's    #
# plotter tool.                                                       #
#######################################################################
# Modules for text interpretation and math
import os, sys, re, fnmatch
import numpy as np
# Modules for plotting and reading csv
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
import csv
%matplotlib inline
from ipywidgets import interactive
# Module for GUI
import tkinter as tk
from tkinter import filedialog
# Module for saving plots as PDF
# from matplotlib.backends.backend_pdf import PdfPages

## Load XML files

In [6]:
# Function for picking XML file order
def csvItemize(text):
    # Regular expressions for naming convention of shots
    # recorded by the scanner
    pattern = '\w+\.csv\.(\d+)'

    # Identify the time point of each shot/time-point recorded
    token = re.search(pattern, text)
    return int(token.group(1)) if token else '&'

def csvSort(order,fileList):
    idxNList = zip(order, fileList)
    sortedList = [x for x in idxNList]
    sortedList.sort(key=lambda sortedList:sortedList[0])
    return list(sortedList)

class UIGetDir(tk.Frame):
    # initialize tkinter object
    def __init__(self, master=None):
        super().__init__(master)
        self.master = master
        self.pack()
        
        self.create_widgets()
        self.info = self.getFileInfo()
        
#         self.master.destroy
        
    def create_widgets(self):
        self.master.geometry("200x100")
        self.message = tk.Message(self)
        self.message["text"] = \
        "Please select a directory with the shots of the sequence then\n close this window"
#         self.message["takefocus"] = True
        self.message["fg"] = "black"
        self.message.pack(side="top")
    
    def getFileInfo(self):
        # starting from user's home folder, pick directory in which XMLs exist
        self.master.sourceFolder = tk.filedialog.askdirectory(parent=self.master,
                                                             initialdir=os.environ['HOME'],
                                                             title="XML directory")
        print(self.master.sourceFolder)
        
        return self.master.sourceFolder

### Pick the location of the XML

In [15]:
#if len(sys.argv) != 5:
#    sys.exit('Usage: python t1rProcessing.py <spinLockImLocation> minTSL maxTSL numTSL')

#print 'Script Name: ', sys.argv[0]
#print 'Input File: ', sys.argv[1]
#print 'Minimum TSL: ', sys.argv[2]
#print 'Maximum TSL: ', sys.argv[3]
#print 'TSL Quantity: ', sys.argv[4]

#min_TSL = sys.argv[2]
#max_TSL = sys.argv[3]
#tslPlayed = sys.argv[4]

# Call GUI window for selection of XML directories
# for storage of their full paths
# uiObj = tk.Tk()
# file = UIGetDir(master=uiObj)
# file.mainloop
csvList = []
# xmlList.append(file.info)

csvList = ["/Users/nowusu/uiHackyHour"]

In [16]:
# Create placeholder and fill list for xmls
tempList = []
csvFiles = []

# provide the absolute path of the current directory
imLoc = xmlList[0] + '/'
# list all files in the CSV directory
dirList = os.listdir(imLoc)
# store only DICOM files
filesInDir = fnmatch.filter(dirList,'*.csv')

for j,k in enumerate(filesInDir):
    tempList.append(xmlItemize(k))
    csvFiles.append(k)

sortedFiles = csvSort(tempList, csvFiles)

# Array for the sorted abs. path for each XML image
csvFullPath = []

# Fill array with XML abs. paths
stopCond = len(sortedFiles)
for i,j in sortedFiles:
    csvFullPath.append(csvList[0] + '/' + j)

    
# Garbage collection of unused variables
del tempList, csvList, sortedFiles

In [46]:
csvFullPath
with open (csvFullPath[0], newline='') as csvfile:
    spamreader = csv.reader(csvfile, dialect='excel')
    for row in spamreader:
        print(', '.join(row))

sample, Condition, X1.Octadecanol, X2.Hydroxybutyrate, X2.Hydroxyglutarate, X2.Oxoadipate, X3.Hydroxypyruvate, X3.Phosphoglycerate, X6.Phosphogluconate, Aconitate, Adenine, Adenosine, Adonitol, Alanine, alpha.Ketoglutarate, alpha.Ketoisocaproate..KIC., Aminoadipate, Arachidic.acid..Eicosanoic.acid., Arachidonate, Asparagine, Aspartate, beta.Alanine, beta.Hydroxy.beta.Methylbutyric.acid..HMB., beta.Hydroxybutyrate..3.Hydroxybutyrate., Cholesterol, Citrate, Citrulline, Cysteine, Cytidine, Cytosine, Deoxyuridine, Dihydroxyacetone.phosphate..DHAP., Dihydroxyphenylalanine..DOPA., Dopamine..3.Hydroxytyramine.HCl., Fructose, Fructose.6.phosphate, Fumarate, Gamma.aminobutyric.acid..GABA., Glucose, Glucose.6.phosphate, Glutamate, Glutamine, Glycerate, Glycerol, Glycerol.Monolaurate..GML., Glycine, Guanine, Guanosine, Heneicosylic.acid..Heneicosanoic.acid., Heptadecanoic.acid, Histidine, Homocysteine, Homoserine, Hypotaurine, Hypoxanthine, Inosine, Inotisol, Isocitrate, Isoleucine, Lactate, Laur

In [48]:
with open(csvFullPath[0], newline='') as csvfile2:
    spamreader2 = csv.DictReader(csvfile2)
    for row_2 in spamreader2:
        print(row_2['sample'],row_2['Condition'])

6 Isradipine
22 Isradipine
14 Isradipine
10 Isradipine
18 Isradipine
16 Isradipine+Rotenone
8 Isradipine+Rotenone
20 Isradipine+Rotenone
24 Isradipine+Rotenone
11 Rotenone
3 Rotenone
19 Rotenone
15 Rotenone
23 Rotenone
7 Rotenone
1 Vehicle
17 Vehicle
9 Vehicle
25 Vehicle
13 Vehicle
5 Vehicle


## Load files

In [29]:
# load an XML for info on first plotter time-point
def csvRead(csvSets, numOfShots):
    ''' Takes: 
            list of XML global addresses (xmlSets),
            file count representing the shots
            acquired (numOfShots).
        Returns: 
            sequencer information for each shot (root).'''
    
    # Sequencer information can be learnt from the 
    # code below.
    # for child in root:
    #    print(child.tag, child.attrib)
    # "root" here is from having the csv instance
    # call the getroot() function.

    # Placeholders for CSV reader objects (tree) and
    # the Sequencer section header (root) info
    fullData = []
    root = [[] for x in range(numOfShots)]
    
    # Fill placeholder with ElementTree objects
    # which represent section headers
    for x,y in enumerate(csvSets):
        print(y)
        fullData.append(csv.reader(y, dialect='excel'))
    
    # Fill placeholder with data for each Sequencer 
#     for i,j in enumerate(fullData):
#         root[i] = j.getroot()
    
#     del fullData
    return fullData

wvm = csvRead(csvFullPath,stopCond)

seqr = {0: 'SSP', 1: 'XGRAD', 2: 'YGRAD', 3: 'ZGRAD',\
        4: 'RHO1', 5: 'RHO2', 6: 'THETA1', 7: 'THETA2'}

/Users/nowusu/uiHackyHour//20190723_chronicrotenoneisradipine_driftnorm.csv
/Users/nowusu/uiHackyHour//Reformated_00163_ET_MN_20190710_ISQ.csv


In [37]:
wvm[0].dialect.delimiter

','

In [38]:
wvm[0].dialect.doublequote

1

In [39]:
wvm[0].dialect.escapechar

In [40]:
wvm[0].dialect.lineterminator

'\r\n'

In [41]:
wvm[0].dialect.quotechar

'"'

In [42]:
wvm[0].dialect.quoting

0

In [43]:
wvm[0].dialect.skipinitialspace

0

In [44]:
wvm[0].dialect.strict

0

In [ ]:
wvm[0].dialect.

In [28]:
for u in wvm[1]:
    print(u)

['/']
['U']
['s']
['e']
['r']
['s']
['/']
['n']
['o']
['w']
['u']
['s']
['u']
['/']
['u']
['i']
['H']
['a']
['c']
['k']
['y']
['H']
['o']
['u']
['r']
['/']
['/']
['R']
['e']
['f']
['o']
['r']
['m']
['a']
['t']
['e']
['d']
['_']
['0']
['0']
['1']
['6']
['3']
['_']
['E']
['T']
['_']
['M']
['N']
['_']
['2']
['0']
['1']
['9']
['0']
['7']
['1']
['0']
['_']
['I']
['S']
['Q']
['.']
['c']
['s']
['v']


In [8]:
def extractWvfm(waveObjs, seq, numOfShots):
    ''' Takes: 
            list of sequencer ElementTree objects for
            each time point (waveObjs),
            Sequencer name (seq),
            and XML file count (numOfShots).
        Returns: 
            Numpy array with abscissa and range of a
            particular Sequencer for all time points.'''
    
    # Placeholder of the data points for each Sequencer.
    oneDimStore = [[] for x in range(numOfShots)]
    twoDimStore = []
    
    # Use splitlines utility to interperate whitespace
    # notation in waveform values for a particular Sequencer.
    # Fill nested list with time/amplitude data for each time point.
    t = 0
    while t != numOfShots:
        for x in waveObjs[t][seq][0].itertext():
            oneDimStore[t].append(x.splitlines())
        t += 1

    # Placeholder for time and amplitude of the Sequencer at
    # each time point.
    waveToPlot = np.zeros((numOfShots,2,len(oneDimStore[0][0])-1))
    
    # Use split utility to place time (abscissa) and amplitude
    # (range) values into separate rows for each time point.
    t = 0
    while t != numOfShots:
        for x,y in enumerate(oneDimStore[t][0]):
            twoDimStore.append(y.split(' '))
        for x in range(1,len(twoDimStore)):
            waveToPlot[t][0][x-1] = float(twoDimStore[x][0])
            waveToPlot[t][1][x-1] = float(twoDimStore[x][1])
        twoDimStore.clear()
        t += 1
           
    del oneDimStore, twoDimStore
    return waveToPlot

## Plot extracted sequencer waveform

### Procedures for comparing waveforms

In [9]:
## find from the end of an array the contiguous time-point that is < 799.0
def sspStopTime(waveObjs,numOfShots):
    ''' Takes:
            list of sequencer ElementTree objects for
            each time point (waveObjs),
            XML file count (numOfShots)
        Returns:
            list of first time points 
            > 799.0 for each shot (lTime)'''
    
    # Waveform of the SSP board
    sspWv = extractWvfm(waveObjs, 0, stopCond)
    
    # Storage for true final time points
    lTime = []
    
    for t in range(0,numOfShots):
        
        # copy SSP waveform for each shot
        iterTime = sspWv[t][0][:].copy()
        
        # last time point for the sequencer
        # this value can be 799.0
        lTimeStamp = sspWv[t][0][-1]
        
        # take the 2nd through 5th to last time point
        # and iterate over them for comparison
        for i,j in enumerate(iterTime.flat[-2:-6:-1]):
            if (j < lTimeStamp) & ((lTimeStamp % j) > 1.0):
                lTime.append(j)
            else:
                continue

            lTimeStamp = j
            
    return lTime

def scaleTime(wave,sspEndTimes,numOfShots):
    ''' Takes:
            waveform of a sequencer (wave),
            vector of end times < 799.0 from the SSP 
            board (sspEndTimes),
            XML file count (numOfShots)
        Returns:
            waveform of a sequencer with end time points
            matching that of the SSP board (modifiedWave),
            count of the number of indices greater than 799.0'''
    
    # Length of input waveform
    waveLen = len(wave[0][0][:])
    
    # Placeholder for index of time values >= 799.0
    idx2Cut = int()
    
    # Count and store the number of time values
    # >= 799.0. This code assumes the count is
    # consistant accross time.
    for t in range(0,numOfShots):
        
        # copy waveform for each shot
        iterTime = wave[t][0][:].copy()
        
        # last time point for the sequencer
        # this value can be 799.0
        lTimeStamp = wave[t][0][-1]
        count = 0
        
        # take the 2nd through 5th to last time point
        # and iterate over them for comparison
        for i,j in enumerate(iterTime.flat[-2:-6:-1]):
            
            if (j < lTimeStamp) & ((lTimeStamp % j) > 1.0):
                
                # replace the first time point most different 
                # from the subsequent one with another.
                wave[t][0][waveLen - 1 - i] = sspEndTimes[t]
                break
            else:
                
                # add to count if the present time point is 
                # not much different from the one before
                count += 1
                continue
            
            lTimeStamp = j
            
        idx2Cut = count
        
    modifiedWave = wave.copy()
            
    return modifiedWave, idx2Cut

def waveTruncate(wave,cols2Cut,numOfShots):
    ''' Takes:
            waveform of a sequencer (wave),
            number of time values >= 799.0 (cols2Cut),
            XML file count (numOfShots)
        Returns:
            truncated waveform of a sequencer (truncatedWv)'''
    
    # length of input waveform
    waveLen = len(wave[0][0][:])
    
    # Placeholder with compensation for
    # decreased sizes of the input waveform
    truncatedWv = np.zeros((numOfShots,2,waveLen-cols2Cut))

    for t in range(0,numOfShots):
        iterTime = wave[t][0][:].copy()
        lTimeStamp = wave[t][0][-1]
        truncatedWv[t] = np.delete(wave[t],range(waveLen-cols2Cut,waveLen),axis=1)
            
    return truncatedWv

### Show waveforms on the same time scale

In [10]:
def waveToPlot(wave,t):
    x = wave[t,0,:]
    y = wave[t,1,:]
    
    return x, y

In [11]:
seqName = [('SSP', 0),('XGRAD', 1),('YGRAD', 2),('ZGRAD', 3),\
           ('RHO1', 4),('RHO2', 5),('THETA1', 6),('THETA2', 7)]

seqs2Show = [0,1,2,3,4,7]; l = len(seqs2Show)
lastSspTimes = sspStopTime(wvm, stopCond)


def playSequencer(shots):
    seqPlots = plt.figure(num=1,figsize=[15.0,27.0])
    
    for i,p in enumerate(seqs2Show):
        npStore = extractWvfm(wvm, p, stopCond)

        wv, toCut = scaleTime(npStore,lastSspTimes,stopCond)

        xtr = waveTruncate(wv,toCut,stopCond)
        
        xVal, yVal = waveToPlot(xtr,shots)
        
        subPlt = seqPlots.add_subplot(l,1,i+1)
        subPlt.plot(xVal,yVal,'b-')
        subPlt.set_xlabel('Time (us)')
        subPlt.set_ylabel('Amplitude (a.u.)')
        subPlt.autoscale(enable=True,axis='x')
        subPlt.set_title('Sequence {0} Board'.format(seqr[p]))
        seqPlots.subplots_adjust(hspace=0.5)
        
        plt.show

### Interactive plots

In [12]:
interactive_plot = interactive(playSequencer, shots=(0, stopCond-1))
output = interactive_plot.children[-1]
output.layout.height = '1650px'
interactive_plot

interactive(children=(IntSlider(value=9, description='shots', max=19), Output(layout=Layout(height='1650px')))…

# Saving off all shots

In [ ]:
# with PdfPages('20shotDwiEPI2.pdf') as pdf:
    
#     for shots in range(stopCond):
    
#         seqPlots = plt.figure(num=shots,figsize=[15.0,27.0])
        
#         for i,p in enumerate(seqs2Show):
            
#             npStore = extractWvfm(wvm, p, stopCond)

#             wv, toCut = scaleTime(npStore,lastSspTimes,stopCond)

#             xtr = waveTruncate(wv,toCut,stopCond)

#             xVal, yVal = waveToPlot(xtr,shots)

#             subPlt = seqPlots.add_subplot(l,1,i+1)
#             subPlt.plot(xVal,yVal,'b-')
#             subPlt.set_xlabel('Time (us)')
#             subPlt.set_ylabel('Amplitude (a.u.)')
#             subPlt.autoscale(enable=True,axis='x')
#             subPlt.set_title('Sequence {0} Board'.format(seqr[p]))
#             seqPlots.subplots_adjust(hspace=0.25)

#         pdf.savefig(seqPlots)
#         plt.close(seqPlots)

In [ ]:
# test_ssp = extractWvfm(wvm, 2, stopCond)
# print('last 5 vals of {0}: {1}\n'.format(ssptest_ssp[1][0][-1:-6:-1]))
# print('last SSP times: {0}\n'.format(lastSspTimes))
# test_wv, test_idx = scaleTime(test_ssp,lastSspTimes,stopCond)
# print('new last 5 vals of SSP: {0}\n'.format(test_ssp[0][0][-1:-6:-1]))
# print('new last 5 vals of SSP: {0}\n'.format(test_ssp[0][1][-1:-6:-1]))
# test_xtr = waveTruncate(test_wv,test_idx,stopCond)
# test_x, test_y = waveToPlot(test_xtr,0)
# plt.plot(test_x,test_y)

## GUI for zooming in on plots

In [ ]:
"""This snippet of code was copied from 
https://stackoverflow.com/questions/3877774/updating-a-graphs-coordinates-in-matplotlib

Additional help for this section should be taken from this YouTube channel
https://www.youtube.com/watch?v=Zw6M-BnAPP0
"""
class guiPlotter(tk.Tk):
    def __init__(self,parent):
        tk.Tk.__init__(self,parent)
        self.parent = parent

        self.protocol("WM_DELETE_WINDOW", self.dest)
        self.main()
        
    def main(self):
        self.fig = plt.figure()
        self.fig = plt.figure(figsize=(3.5,3.5))

        self.frame = tk.Frame(self)
        self.frame.pack(padx=15,pady=15)

        self.canvas = FigCanvas(self.fig, master=self.frame)

        self.canvas.get_tk_widget().pack(side='top', fill='both')

        self.canvas._tkcanvas.pack(side='top', fill='both', expand=1)

        self.toolbar = NavTb2( self.canvas, self )
        self.toolbar.update()
        self.toolbar.pack()

        self.btn = Tkinter.Button(self,text='button',command=self.alt)
        self.btn.pack(ipadx=250)

        self.draw_sphere()
        
    def alt (self):
        self.draw_sphere(5)
        
    def draw_sphere(self, prop=10):
        self.fig.clear()
        ax = Axes3D(self.fig)

        u = np.linspace(0, 2 * np.pi, 100)
        v = np.linspace(0, np.pi, 100)

        x = prop * np.outer(np.cos(u), np.sin(v))
        y = prop * np.outer(np.sin(u), np.sin(v))
        z = prop * np.outer(np.ones(np.size(u)), np.cos(v))

        t = ax.plot_surface(x, y, z, rstride=4, cstride=4,color='lightgreen',linewidth=0)
        self.canvas.draw()